<a href="https://colab.research.google.com/github/ianog88/Statistics_DataScience/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data using yfinance API

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [ ]:
df = yf.download(tickers=['AAPL'], period="60d", interval="5m")

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-10 09:30:00-05:00,474.160004,474.850006,474.140015,474.809998,474.809998,2441239
2024-01-10 09:35:00-05:00,474.820007,475.179993,474.760010,475.140015,475.140015,1161309
2024-01-10 09:40:00-05:00,475.149994,475.339996,474.869995,475.160004,475.160004,889787
2024-01-10 09:45:00-05:00,475.165009,475.250000,474.850006,475.089996,475.089996,826248
2024-01-10 09:50:00-05:00,475.079987,475.149994,474.440002,474.464996,474.464996,756481
...,...,...,...,...,...,...
2024-04-05 15:35:00-04:00,518.950012,518.955017,518.169983,518.440002,518.440002,714994
2024-04-05 15:40:00-04:00,518.429993,518.585022,518.140015,518.184998,518.184998,624457
2024-04-05 15:45:00-04:00,518.179993,518.719971,518.119995,518.460022,518.460022,1145878


# Perform some feature engineering, reset the index and remove some missing values

In [ ]:
df['return'] = (df['Open'] - (df['Open'].shift())) / (df['Open'].shift())
df['limit'] = ((df['Low'] - df['Open']) / df['Open']).shift()
df['gap'] = (df['Open'] - (df['Adj Close'].shift())) / (df['Adj Close'].shift())

In [ ]:
df.reset_index(inplace=True)
df['date'] = df['Datetime'].dt.strftime('%m-%d')
df['hour'] = df['Datetime'].dt.strftime('%r')

In [ ]:
df = df[78:4673]

In [ ]:
df = df[df.date != '03-04']
df = df[df.date != '03-05']
df = df[df.date != '03-06']

In [ ]:
df = df.reset_index()
df = df[['return','date', 'hour', 'gap', 'limit']]

In [ ]:
df.head()

,return,date,hour,gap,limit
0,0.006611,12-28,09:30:00 AM,0.005021,-0.000181
1,0.000567,12-28,09:35:00 AM,0.000121,-0.000773
2,0.000309,12-28,09:40:00 AM,-0.000052,-0.000463
3,0.001673,12-28,09:45:00 AM,0.000000,-0.000437
4,-0.001927,12-28,09:50:00 AM,-0.000026,-0.002672


## We want to create a dataframe where each row represents a time step in the series, as we are using 5 minute data each day is comprised of 79 rows (an opening price and the 78 5min price changes), the dataframe is comprised of 56 days resulting in a total length of 4368 (56x78)

In [ ]:
a = np.zeros((4368,79))

for i in range(4368):
  if(i % 78 == 0):
    a[i,0] = df.iloc[i,3]
  else:
    a[i] = a[i-1]
    a[i,i%78] = df.iloc[i,0]
    a[i,-1] = df.iloc[i,4]


In [ ]:
a_df = pd.DataFrame(a)

# Save the dataframe to google drive for use in the Actor Critic model

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
a_df.to_csv('/content/gdrive/My Drive/AAPL_5min.csv')